#### Vergleichskript
Dieses Skipt vergleicht zwei Refernzzählungen und erzeugt daraus zwei DataFrames mit Fehlern.

In [21]:
import pandas as pd
def compare_dataframes(df1, df2, time_threshold=300):
    """
    Diese Funktion sucht nach ähnlichen Zeilen in den beiden Dataframes basierend auf der 'SectionID' und 'DateTime'.
    Die Zeilen werden als ähnlich betrachtet, wenn die 'SectionID' identisch ist und der 'DateTime'-Unterschied kleiner ist als der
    angegebene 'time_threshold' in Sekunden.

    :param df1: Erstes DataFrame
    :param df2: Zweites DataFrame
    :param time_threshold: Zeitschwellenwert in Sekunden (Standardwert: 300)
    :return: Ein Tuple mit zwei DataFrames, die die nicht ähnlichen Zeilen der beiden Eingabe-DataFrames enthalten.
    """
    # Zuerst die Datetime-Spalten in datetime64 konvertieren, falls sie noch nicht konvertiert wurden
    df1["DateTime"] = pd.to_datetime(df1["DateTime"])
    df2["DateTime"] = pd.to_datetime(df2["DateTime"])

    # Leere DataFrames erstellen, um nicht ähnliche Zeilen zu speichern
    not_similar_df1 = pd.DataFrame(columns=df1.columns)
    not_similar_df2 = pd.DataFrame(columns=df2.columns)

    for index1, row1 in df1.iterrows():
        similar = False
        for index2, row2 in df2.iterrows():
            if row1["SectionID"] == row2["SectionID"]:
                time_difference = abs((row1["DateTime"] - row2["DateTime"]).total_seconds())
                if time_difference <= time_threshold:
                    similar = True
                    break
        if not similar:
            not_similar_df1 = not_similar_df1.append(row1)

    for index2, row2 in df2.iterrows():
        similar = False
        for index1, row1 in df1.iterrows():
            if row1["SectionID"] == row2["SectionID"]:
                time_difference = abs((row1["DateTime"] - row2["DateTime"]).total_seconds())
                if time_difference <= time_threshold:
                    similar = True
                    break
        if not similar:
            not_similar_df2 = not_similar_df2.append(row2)

    return not_similar_df1, not_similar_df2


In [40]:
gt =  pd.read_csv("./data/gt_data/Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-00_v5.csv")
gtv = pd.read_csv("./data/gtv_data/Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-00_events.csv")

In [41]:
#Berechnet die absolute Zeilendifferenz:
gt.shape[0] - gtv.shape[0]

26

In [42]:
#Einfache Diferenzbetrachtung
gt_f = gt["Class"].value_counts()
gt_f.name = "gt"
gtv_f = gtv["Class"].value_counts()
gtv_f.name = "gtv"
gt_merged = pd.concat([gt_f, gtv_f], axis = 1)
gt_merged.fillna(value=0, inplace=True)
gt_merged["diff"] = gt_merged["gt"] - gt_merged["gtv"]
gt_merged["rel"] = gt_merged["diff"] / gt_merged["gt"] * 100
gt_merged["prio"] = abs(gt_merged["rel"]) * gt_merged["gt"] / gt.shape[0]
gt_merged.sort_values("prio", ascending= False )
#Anmerkung: Wenn positiv dann hat gt mehr gezählt als gtv  

,gt,gtv,diff,rel,prio
Person,553,540,13,2.350814,1.272016
Pkw ohne Anhänger,350,344,6,1.714286,0.587084
Fahrrad ohne Anhänger,41,38,3,7.317073,0.293542
Lieferwagen ohne Anhänger,42,40,2,4.761905,0.195695
Lkw ohne Anhänger,4,2,2,50.000000,0.195695
Bus,18,18,0,0.000000,0.000000
Motorisierte Zweiräder,10,10,0,0.000000,0.000000
Lkw mit Anhänger,2,2,0,0.000000,0.000000
Lieferwagen mit Anhänger,2,2,0,0.000000,0.000000


In [61]:
#Vergleichsklasse
comp = "Lieferwagen ohne Anhänger"
gt_diff, gtv_diff = compare_dataframes(gt[gt["Class"] == comp], gtv[gtv["Class"] == comp], 2)
#Fehlermeldung wird Ausgegeben aber code funktioniert!

C:\Users\sebas\AppData\Local\Temp\ipykernel_15152\3064643632.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["DateTime"] = pd.to_datetime(df1["DateTime"])
C:\Users\sebas\AppData\Local\Temp\ipykernel_15152\3064643632.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["DateTime"] = pd.to_datetime(df2["DateTime"])
C:\Users\sebas\AppData\Local\Temp\ipykernel_15152\3064643632.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [64]:
#1. Tabelle mit problematischen Events in der Referenzzählung
gt_diff

,EventID,SectionID,TrackID,X,Y,Frame,Class,Vidfilename,DateTime
536,552,Haltelinie,277,269,292,11384,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:24:29
537,553,Querschnitt_Straße,277,629,453,11412,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:24:30
568,584,Haltelinie,293,281,287,12105,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:25:05
569,585,Querschnitt_Straße,293,647,428,12129,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:25:06
668,684,Haltelinie,342,375,258,14089,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:26:44
1014,1031,Haltelinie,532,277,290,7770,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:21:28
1015,1032,Querschnitt_Straße,532,646,449,7786,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:21:29


In [65]:
#2. Tabelle mit problematischen Events in der alternativen Referenzzählung
gtv_diff

,EventID,SectionID,TrackID,X,Y,Frame,Class,Vidfilename,DateTime
112,132,Haltelinie,68,388,250,5028,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:19:11
113,133,Querschnitt_Straße,68,733,368,5060,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:19:13
234,254,Haltelinie,129,385,253,10518,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:23:45
235,255,Querschnitt_Straße,129,735,368,10558,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:23:47
406,426,Haltelinie,215,384,254,17485,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:29:34
407,427,Querschnitt_Straße,215,744,365,17515,Lieferwagen ohne Anhänger,Saarbruecken_OTCamera04_FR20_2022-10-17_13-15-...,2022-10-17 13:29:35
